# Import packages

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pandas as pd
import regex as re

apikey = '' # an api key can be requested via the dataservices of the 
            #Koninkijke Bibliotheek, see https://www.kb.nl/bronnen-zoekwijzers/dataservices-en-apis

# Retrieve identifiers

In [ ]:
# %20 is a space in the query url
# see https://developer.mozilla.org/en-US/docs/Glossary/percent-encoding

dfIdentifiers = pd.DataFrame(columns = ['identifier', 'type', 'title', 'papertitle', 'date'])

start_date = "1618-01-01" #YYYY-MM-DD
end_date = "1791-12-31" # Change start and end date for different subsets of the delpher archive
file_name = start_date[:4]+"_"+end_date[:4]+".csv"
dates = start_date[:4]+"_"+end_date[:4]

subject = f"%28universiteit%20OR%20universiteiten%20OR%20universitair%20OR%20vrouwen%20OR%20vrā%20OR%20vrau%20OR%20vrauwen%20OR%20vrouwen%20OR%20vroutje%20OR%20vrouw%20OR%20vrouwe%20OR%20vrouws%20OR%20vrouwtjens%20OR%20onderwijs%20OR%20onderwys%20OR%20Aletta_Jacobs%20OR%20vrouwenkiesrecht%20OR%20betaald_werk%20OR%20Vrije_Vrouwenvereeniging%20OR%20Vereeniging_voor_Vrouwenkiesrecht%20OR%20VVK%20OR%20Wilhelmina_Drucker%20OR%20vrouwenarbeid%20OR%20gezinsloon%20OR%20huishouden%20OR%20huwelijk%20OR%20seksualiteit%20OR%20kiesrecht%20OR%20moeder%20OR%20moederschap%20OR%20sufragettes%20OR%20feminisme%20OR%20vrouwenbeweging%20OR%20vrouwenkiesrecht%20OR%20vrouwenstemrecht%20OR%20vrouwenkiesrechtstrijd%20OR%20vrouwenvraagstuk%20OR%20vrouwenemancipatie%20OR%20vrouwentekens%20OR%20yoni_teken%20OR%20yoni%20OR%20Dolle_Mina%20OR%20abortus%20OR%20vrouwenstaking%20OR%20vrouwenpraatgroepen%20OR%20emancipatie%20OR%20emancipatiebeleid%20OR%20vrouwenonderdrukking%20OR%20huishoudelijke_arbeid%20OR%20huisvrouwen%20OR%20huisvrouwensyndroom%20OR%20voscursussen%20OR%20amazonen%20OR%20heksen%29AND%28date%20within%20%22{start_date}%20{end_date}%22%29"
startRecord = 0 #The first record number you want to retrieve from your query

#query needs to be expressed in CQL syntax
query = f"""http://jsru.kb.nl/sru/sru/{apikey}?query={subject}&x-collection=DDD_artikel&recordSchema=dc&startRecord={str(startRecord)}&maximumRecords=1000"""
page = requests.get(query)
soup = BeautifulSoup(page.content, "xml")
for item in soup.findAll('srw:searchRetrieveResponse'):
    records = item.find('srw:numberOfRecords').text
    
print("Total records available:",records)

records = 200000 # The maximum amount of articles you want to retrieve + 1000.

while startRecord <= int(records):
    query = f"""http://jsru.kb.nl/sru/sru/{apikey}?query={subject}&x-collection=DDD_artikel&recordSchema=dc&startRecord={str(startRecord)}&maximumRecords=1000"""
    page = requests.get(query)
    soup = BeautifulSoup(page.content, "xml")
    
    for item in soup.findAll('srw:recordData'):
        identifier = item.find('dc:identifier')
        kind =  item.find('dc:type')    
        title = item.find('dc:title')
        papertitle = item.find('dc:publisher')
        date = item.find('dc:date')
        if title is None:
            title = ""            
            dfIdentifiers = dfIdentifiers.append({'identifier': identifier.text, 'type': kind.text, 'title': title, 'papertitle': papertitle.text, 'date': date.text}, ignore_index = True)
        else:
            dfIdentifiers = dfIdentifiers.append({'identifier': identifier.text, 'type': kind.text, 'title': title.text, 'papertitle': papertitle.text, 'date': date.text}, ignore_index = True)
    
    startRecord = startRecord + 1000
    print(f"Amount of records retrieved: {startRecord}")

Total records available: 83883
Amount of records retrieved: 1000
Amount of records retrieved: 2000
Amount of records retrieved: 3000
Amount of records retrieved: 4000
Amount of records retrieved: 5000
Amount of records retrieved: 6000
Amount of records retrieved: 7000
Amount of records retrieved: 8000
Amount of records retrieved: 9000
Amount of records retrieved: 10000
Amount of records retrieved: 11000
Amount of records retrieved: 12000
Amount of records retrieved: 13000
Amount of records retrieved: 14000
Amount of records retrieved: 15000
Amount of records retrieved: 16000
Amount of records retrieved: 17000
Amount of records retrieved: 18000
Amount of records retrieved: 19000
Amount of records retrieved: 20000
Amount of records retrieved: 21000
Amount of records retrieved: 22000
Amount of records retrieved: 23000
Amount of records retrieved: 24000
Amount of records retrieved: 25000
Amount of records retrieved: 26000
Amount of records retrieved: 27000
Amount of records retrieved: 2800

In [ ]:
len(dfIdentifiers)

83884

In [ ]:
# Check dates
date = dfIdentifiers["date"]

print(date.min(), '\n', date.max())

1619/06/15 00:00:00 
 1791/12/31 00:00:00


In [ ]:
# Save df identifiers
file_path = "Define the location where you want to save your file"
path = file_path+dates+"_identifiers.csv"
dfIdentifiers.to_csv(path_or_buf=path)

# Retrieve texts

In [ ]:
print("NA values in identifiers:",dfIdentifiers['identifier'].isna().any())
print("Duplicate values in identifiers:",dfIdentifiers['identifier'].duplicated().any())

# Remove duplicates
dfIdentifiers = dfIdentifiers.drop_duplicates(subset=['identifier'])
print("Duplicate values in identifiers:",dfIdentifiers['identifier'].duplicated().any())

# Check amount of unique identifiers
print("Amount of unique identifiers:",len(dfIdentifiers['identifier']))

NA values in identifiers: False
Duplicate values in identifiers: True
Duplicate values in identifiers: False
Amount of unique identifiers: 83883


In [ ]:
dfTekst = pd.DataFrame(columns = ['identifier', 'text'])

In [ ]:
for index, row in dfIdentifiers.iterrows():
    if index % 1000 == 0: #Get a print statement every 1000 items
        print("Retrieved first ",index," texts.")
    
    identifier = row['identifier']
    url = requests.get(identifier)
    soup = BeautifulSoup(url.content, "xml")
    text = ''
    for item in soup.findAll('p'): #this will find all text within XML tags with p: soup.findAll('p'):[<p>one</p>, <p>two</p>] see beatofulsoup documentation
        text = text + (item.text)

    dfTekst = dfTekst.append({'identifier': identifier, 'text': text}, ignore_index = True)
    
'''beatiful soup documentation:
https://www.crummy.com/software/BeautifulSoup/bs3/documentation.html#The%20basic%20find%20method%3a%20findAll%28name,%20attrs,%20recursive,%20text,%20limit,%20%2a%2akwargs%29 '''

Retrieved first  0  texts.
Retrieved first  2000  texts.
Retrieved first  3000  texts.
Retrieved first  4000  texts.
Retrieved first  5000  texts.
Retrieved first  6000  texts.
Retrieved first  7000  texts.
Retrieved first  8000  texts.
Retrieved first  9000  texts.
Retrieved first  10000  texts.
Retrieved first  11000  texts.
Retrieved first  12000  texts.
Retrieved first  13000  texts.
Retrieved first  14000  texts.
Retrieved first  15000  texts.
Retrieved first  16000  texts.
Retrieved first  17000  texts.
Retrieved first  18000  texts.
Retrieved first  19000  texts.
Retrieved first  20000  texts.
Retrieved first  21000  texts.
Retrieved first  22000  texts.
Retrieved first  23000  texts.
Retrieved first  24000  texts.
Retrieved first  25000  texts.
Retrieved first  26000  texts.
Retrieved first  27000  texts.
Retrieved first  28000  texts.
Retrieved first  29000  texts.
Retrieved first  30000  texts.
Retrieved first  31000  texts.
Retrieved first  32000  texts.
Retrieved first  330

'beatiful soup documentation:\nhttps://www.crummy.com/software/BeautifulSoup/bs3/documentation.html#The%20basic%20find%20method%3a%20findAll%28name,%20attrs,%20recursive,%20text,%20limit,%20%2a%2akwargs%29 '

In [ ]:
len(dfTekst)

83883

# Merge dataframes

In [ ]:
# Merge text and identifier dataframes
dfIdentifiers = dfIdentifiers.merge(dfTekst, on = 'identifier', how = 'inner')
dfIdentifiers.head(4)

,identifier,type,title,papertitle,date,text
0,http://resolver.kb.nl/resolve?urn=ddd:01079908...,artikel,PORTUGAL.,Johannes Enschede en Zoonen,1775/05/04 00:00:00,"Lissabon den 4 April, 't Is zeer gemeen, en me..."
1,http://resolver.kb.nl/resolve?urn=ddd:01019858...,artikel,PORTUGAL.,"Callenfels, Mandelgreen en Taillefert",1775/05/09 00:00:00,"Lissabon, deni 4 April. Mszeer femeen,et menri..."
2,http://resolver.kb.nl/resolve?urn=ddd:01010643...,artikel,ZWEEDEN.,C. Barlinckhoff [etc];A.S. Hoitsema,1775/12/01 00:00:00,"STOKKOLM den 6 November. Deezer dagen , heeft ..."
3,http://resolver.kb.nl/resolve?urn=ddd:01072136...,advertentie,Advertentie,Jan Verlem,1785/09/05 00:00:00,)?.etêr î-tiffetij Sfitts ftict —• — na Noorwe...


In [ ]:
len(dfIdentifiers)

83883

# Preprocess dataframe

In [ ]:
df = dfIdentifiers

## remove duplicate entries based on the text column
df = df.drop_duplicates(subset=['text'])

## remove unnecessary columns
df = df.drop(['Unnamed: 0'], axis=1, errors='ignore')

## reset the index
df = df.reset_index()
print(df.info())

## check for NA in text & drop
print("NA values in text:",df['text'].isna().any())
print(type(df))
df = df.dropna(axis=0, how='any', subset=['text'])
print("NA values in text:",df['text'].isna().any())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83883 entries, 0 to 83882
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       83883 non-null  int64 
 1   identifier  83883 non-null  object
 2   type        83883 non-null  object
 3   title       83883 non-null  object
 4   papertitle  83883 non-null  object
 5   date        83883 non-null  object
 6   text        83883 non-null  object
dtypes: int64(1), object(6)
memory usage: 4.5+ MB
None
NA values in text: False
<class 'pandas.core.frame.DataFrame'>
NA values in text: False


# Save final dataframe

In [ ]:
# Save dataframe to csv
path = file_path+file_name
df.to_csv(path_or_buf=path)